In [4]:
import streamlit as st
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd
import pandas_gbq as pdb
import tqdm

# Create API client.
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
client = bigquery.Client(credentials=credentials)


In [ ]:
# ### Download das bases
# #Dicionario
# df_dicionario = pdb.read_gbq("select * from basedosdados.br_ibge_pnad_covid.dicionario", project_id="postech-414800", credentials=credentials)
# df_dicionario.to_csv('.\\bases\\dicionario.csv',sep=";", header=True)

# #microdados
# df_ano_mes = pdb.read_gbq("select ano, mes, count(*) as cnt from basedosdados.br_ibge_pnad_covid.microdados group by ano, mes", project_id="postech-414800", credentials=credentials)
# df_ano_mes.sort_values(by=['mes'], inplace=True)

# lst_df = []
# for k,i in df_ano_mes.iterrows():
#     print(f"{i['ano']} - {i['mes']}")
#     df_tmp = pdb.read_gbq(f"select * from basedosdados.br_ibge_pnad_covid.microdados where ano = {i['ano']} and mes = {i['mes']}", project_id="postech-414800", credentials=credentials)
#     lst_df.append(df_tmp)

# df_concat = pd.concat(lst_df)
# df_concat.to_csv('.\\bases\\microdados.csv',sep=";", header=True)

In [6]:
df_dic = pd.read_csv(r'.\bases\dicionario.csv', sep=";")
df_m = pd.read_csv(r'.\bases\microdados.csv', sep=";")

,Unnamed: 0,ano,mes,semana,sigla_uf,capital,rm_ride,estrato,upa,v1008,...,f001,f0021,f0022,f002a1,f002a2,f002a3,f002a4,f002a5,f0061,f006
0,0,2020,5,2,AP,16.0,16.0,1610111,160000029,2,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0
1,1,2020,5,2,AP,16.0,16.0,1610111,160000029,2,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0
2,2,2020,5,2,AP,16.0,16.0,1610111,160000029,4,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0
3,3,2020,5,2,AP,16.0,16.0,1610111,160000029,4,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0
4,4,2020,5,2,AP,16.0,16.0,1610111,160000029,4,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650454,381433,2020,11,1,MG,NaN,NaN,3154012,310417255,13,...,1,NaN,NaN,1.0,1.0,1.0,2.0,1.0,1,1.0
2650455,381434,2020,11,1,MG,NaN,NaN,3154012,310417255,13,...,1,NaN,NaN,1.0,1.0,1.0,2.0,1.0,1,1.0
2650456,381435,2020,11,1,MG,NaN,NaN,3154012,310417255,13,...,1,NaN,NaN,1.0,1.0,1.0,2.0,1.0,1,1.0
2650457,381436,2020,11,1,MG,NaN,NaN,3154012,310417255,13,...,1,NaN,NaN,1.0,1.0,1.0,2.0,1.0,1,1.0


In [18]:
df_d = df_dic.query("nome_coluna == 'capital'")[['chave','valor']]
df_j = df_m[['ano','mes','semana','capital']].merge(df_d, how='left', left_on='capital', right_on='chave')


In [19]:
df_j

,ano,mes,semana,capital,chave,valor
0,2020,5,2,16.0,16.0,Município de Macapá (AP)
1,2020,5,2,16.0,16.0,Município de Macapá (AP)
2,2020,5,2,16.0,16.0,Município de Macapá (AP)
3,2020,5,2,16.0,16.0,Município de Macapá (AP)
4,2020,5,2,16.0,16.0,Município de Macapá (AP)
...,...,...,...,...,...,...
2650454,2020,11,1,NaN,NaN,NaN
2650455,2020,11,1,NaN,NaN,NaN
2650456,2020,11,1,NaN,NaN,NaN
2650457,2020,11,1,NaN,NaN,NaN


In [20]:
df_dic

,Unnamed: 0,id_tabela,nome_coluna,chave,cobertura_temporal,valor
0,0,microdados,capital,11.0,2020-05(1)2020-11,Município de Porto Velho (RO)
1,1,microdados,capital,12.0,2020-05(1)2020-11,Município de Rio Branco (AC)
2,2,microdados,capital,13.0,2020-05(1)2020-11,Município de Manaus (AM)
3,3,microdados,capital,14.0,2020-05(1)2020-11,Município de Boa Vista (RR)
4,4,microdados,capital,15.0,2020-05(1)2020-11,Município de Belém (PA)
...,...,...,...,...,...,...
549,549,microdados,f002a5,2.0,2020-05(1)2020-11,Não
550,550,microdados,f002a5,3.0,2020-05(1)2020-11,Não sabe
551,551,microdados,f0061,1.0,2020-05(1)2020-11,Pessoa moradora
552,552,microdados,f0061,2.0,2020-05(1)2020-11,Pessoa não moradora


In [9]:
for i in df_dic['nome_coluna'].unique():
    df_dic_tmp = df_dic.query("nome_coluna = '@i'")[[]]


capital
rm_ride
v1022
v1023
a001a
a003
a004
a005
a006
a006a
a006b
a007
a007a
a008
a009
b0011
b0012
b0013
b0014
b0015
b0016
b0017
b0018
b0019
b00110
b00111
b00112
b00113
b002
b0031
b0032
b0033
b0034
b0035
b0036
b0037
b0041
b0042
b0043
b0044
b0045
b0046
b005
b006
b007
b008
b009a
b009b
b009c
b009d
b009e
b009f
b0101
b0102
b0103
b0104
b0105
b0106
b011
c001
c002
c003
c004
c005
c006
c007
c007a
c007b
c007c
c007d
c007e
c007f
c009a
c010
c0101
c01011
c0102
c01021
c0103
c0104
c011a
c011a1
c011a11
c011a2
c011a21
c012
c013
c014
c015
c016
c017a
d0011
d0021
d0031
d0041
d0051
d0061
d0071
e001
e0021
e0022
e0023
e0024
f001
f0022
f002a1
f002a2
f002a3
f002a4
f002a5
f0061
